Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Classification using Automated ML

In this example we use Azure ML's Automated ML functionality to improve on the classifier we built earlier. Automated ML handles the task of building many models from a wide variety of algorithms and choosing a good set of hyper-parameters for them. We then select best the model (or one that meets our criteria) and deploy it as a web service.

## Load and prepare experiment

In [4]:
%run "../includes/mnt_blob"

As part of the setup we have already created an AML workspace. Let's load the workspace and create an experiment.

In [6]:
import json
import logging
import os
import random

from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import pandas as pd
from sklearn import datasets

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun

We load the workspace directly from the config file we created in the early part of the course.

In [8]:
config_path = '/dbfs/tmp/aml_config'
ws = Workspace.from_config(path=os.path.join(config_path, 'config.json'))

experiment_name =  'pred-maint-automl' # choose a name for experiment
project_folder = '.' # project folder

experiment=Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data=output, index=['']).T

Found the config file in: /dbfs/tmp/aml_config/config.json
 Out[ 28 ]: 
 
Experiment Name pred-maint-automl 
Location westus2 
Project Directory . 
Resource Group ODL-ml-49581 
SDK version 1.0.10 
Subscription ID 4c85b753-201a-4e91-9dc7-23b929520091
Workspace myADBworkspace

Opt in for diagnostics for better experience, quality, and security of future releases:

In [10]:
from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on.

## Instantiate config file

We now instantiate a `AutoMLConfig` object. This defines the settings and data used to run the experiment.

|Property|Description|
|-|-|
|**task**|classification or regression|
|**primary_metric**|This is the metric that you want to optimize.<br> Classification supports the following primary metrics <br><i>accuracy</i><br><i>AUC_weighted</i><br><i>balanced_accuracy</i><br><i>average_precision_score_weighted</i><br><i>precision_score_weighted</i>|
|**max_time_sec**|Time limit in seconds for each iterations|
|**iterations**|Number of iterations. In each iteration Auto ML trains the data with a specific pipeline|
|**n_cross_validations**|Number of cross validation splits|
|**X**|(sparse) array-like, shape = [n_samples, n_features]|
|**y**|(sparse) array-like, shape = [n_samples, ], [n_samples, n_classes]<br>Multi-class targets. An indicator matrix turns on multilabel classification.  This should be an array of integers. |
|**path**|Relative path to the project folder.  AutoML stores configuration files for the experiment under this folder. You can specify a new empty folder. |

In [13]:
import azureml.core

print("SDK Version:", azureml.core.VERSION)

SDK Version: 1.0.10

In [14]:
filepath = '/dbfs/mnt/data/telemetry/'

X_train = pd.read_csv(filepath + 'X_train.csv', header=0)
X_test = pd.read_csv(filepath + 'X_test.csv', header=0)
y_train = pd.read_csv(filepath + 'y_train.csv', header=0)
y_test = pd.read_csv(filepath + 'y_test.csv', header=0)

In [15]:
X_train.head()

Out[ 32 ]: 
 volt_ma_3 rotate_ma_3 pressure_ma_3 ... df_3 df_4 age
0 170.028993 449.533798 94.592122 ... 0.0 0.0 18 
1 164.192565 403.949857 105.687417 ... 3.0 3.0 18 
2 168.134445 435.781707 107.793709 ... 6.0 6.0 18 
3 165.514453 430.472823 101.703289 ... 9.0 9.0 18 
4 168.809347 437.111120 90.911060 ... 12.0 12.0 18 

[5 rows x 39 columns]

In [16]:
X_train.columns

Out[ 33 ]: 
Index(['volt_ma_3', 'rotate_ma_3', 'pressure_ma_3', 'vibration_ma_3',
 'volt_sd_3', 'rotate_sd_3', 'pressure_sd_3', 'vibration_sd_3',
 'volt_ma_12', 'rotate_ma_12', 'pressure_ma_12', 'vibration_ma_12',
 'volt_sd_12', 'rotate_sd_12', 'pressure_sd_12', 'vibration_sd_12',
 'e_1', 'e_2', 'e_3', 'e_4', 'e_5', 'de_1', 'de_2', 'de_3', 'de_4',
 'de_5', 'm_1', 'm_2', 'm_3', 'm_4', 'dm_1', 'dm_2', 'dm_3', 'dm_4',
 'df_1', 'df_2', 'df_3', 'df_4', 'age'],
 dtype='object')

In [17]:
y_train.head()

Out[ 34 ]: 
 y_1 y_2 y_3 y_4
0 0 0 0 0 
1 0 0 0 0 
2 0 0 0 0 
3 0 0 0 0 
4 0 0 0 0

Here are the metrics we can choose to optimize our model over.

In [19]:
azureml.train.automl.constants.Metric.CLASSIFICATION_PRIMARY_SET

Out[ 35 ]: 
{'AUC_weighted',
 'accuracy',
 'average_precision_score_weighted',
 'norm_macro_recall',
 'precision_score_weighted'}

We now set up a configuration file for the automated ML training experiment. It contains details for how the experiment should run.

In [21]:
automl_config = AutoMLConfig(task='classification', 
                             preprocess=False,
                             name=experiment_name,
                             debug_log='automl_errors.log',
                             primary_metric='AUC_weighted',
                             iteration_timeout_minutes=5,
                             iterations=3,
                             verbosity=logging.INFO,
                             X = X_train.values, # we convert from pandas to numpy arrays using .vaules
                             y = y_train.values[:, 0], # we convert from pandas to numpy arrays using .vaules
                             X_valid=X_test.values,
                             y_valid=y_test.values[:, 0],
                             path=project_folder)

In [22]:
help(AutoMLConfig)

Help on class AutoMLConfig in module azureml.train.automl.automlconfig:

class AutoMLConfig(builtins.object)
 Configuration for submitting an Automated Machine Learning experiment in Azure Machine Learning service.
 
 This configuration object contains and persists the parameters for configuring
 the experiment run parameters, as well as the training data to be used at run time. The following
 code shows a basic example of creating an AutoMLConfig object, and submitting an
 experiment with the defined configuration:
 
 .. code-block:: python
 
 from azureml.core.experiment import Experiment
 from azureml.core.workspace import Workspace
 from azureml.train.automl import AutoMLConfig
 
 automated_ml_config = AutoMLConfig(task = 'regression',
 X = your_training_features,
 y = your_training_labels,
 iterations=30,
 iteration_timeout_minutes=5,
 primary_metric="spearman_correlation")
 
 ws = Workspace.from_config()
 experiment = Experiment(ws, "your-experiment-name")
 run = experiment.submit(automated_ml_config, show_output=True)
 
 
 :param task: 'classification', 'regression', or 'forecasting' depending on what kind of ML problem to solve.
 :type task: str or azureml.train.automl.constants.Tasks
 :param path: Full path to the Azure Machine Learning project folder.
 :type path: str
 :param iterations:
 Total number of different algorithm and parameter combinations
 to test during an Automated Machine Learning experiment.
 :type iterations: int
 :param data_script: File path to the user authored script containing get_data() function.
 :type data_script: str
 :param primary_metric:
 The metric that Automated Machine Learning will optimize for model selection.
 Automated Machine Learning collects more metrics
 than it can optimize. Please consult azureml.train.automl.constants.Metric.CLASSIFICATION_PRIMARY_SET for
 valid metrics for a classification task and azureml.train.automl.constants.Metric.REGRESSION_PRIMARY_SET for
 valid metrics for a regression task.
 :type primary_metric: str or azureml.train.automl.constants.Metric
 :param compute_target: The Azure Machine Learning compute target to run the
 Automated Machine Learning experiment on.
 See https://docs.microsoft.com/azure/machine-learning/service/how-to-auto-train-remote for more
 information on compute targets.
 :type compute_target: azureml.core.compute.AbstractComputeTarget
 :param spark_context: Spark context, only applicable when used inside Azure Databricks/Spark environment.
 :type spark_context: SparkContext
 :param X: The training features to use when fitting pipelines during an experiment.
 :type X: pandas.DataFrame or numpy.ndarray or azureml.dataprep.Dataflow
 :param y: Training labels to use when fitting pipelines during an experiment.
 This is the value your model will predict.
 :type y: pandas.DataFrame or numpy.ndarray or azureml.dataprep.Dataflow
 :param sample_weight:
 The weight to give to each training sample when running fitting pipelines,
 each row should correspond to a row in X and y data.
 :type sample_weight: pandas.DataFrame or numpy.ndarray or azureml.dataprep.Dataflow
 :param X_valid: validation features to use when fitting pipelines during an experiment.
 :type X_valid: pandas.DataFrame or numpy.ndarray or azureml.dataprep.Dataflow
 :param y_valid: validation labels to use when fitting pipelines during an experiment.
 :type y_valid: pandas.DataFrame or numpy.ndarray or azureml.dataprep.Dataflow
 :param sample_weight_valid:
 The weight to give to each validation sample when running scoring pipelines,
 each row should correspond to a row in X and y data.
 :type sample_weight_valid: pandas.DataFrame or numpy.ndarray or azureml.dataprep.Dataflow
 :param cv_splits_indices:
 Indices where to split training data for cross validation.
 Each row is a separate cross fold and within each crossfold, provide 2 arrays, t
 he first with the indices for samples to use for training data and the second with the indices to
 use for validation data. i.e [[t1, v1], 

## Run training experiment

You can call the submit method on the experiment object and pass the run configuration. For Local runs the execution is synchronous. Depending on the data and number of iterations this can run for while.
You will see the currently running iterations printing to the console.

In [25]:
local_run = experiment.submit(automl_config, show_output = True)

Running on local machine
Parent Run ID: AutoML_f5fa6fa9-6a9f-4bd5-9fd2-b4132e4ac2be
*******************************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
TRAINFRAC: Fraction of the training data to train on.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
*******************************************************************************************************************

 ITERATION PIPELINE TRAINFRAC DURATION METRIC BEST
 0 MaxAbsScaler LightGBM 1.0000 0:00:19 0.9941 0.9941
 1 StandardScalerWrapper LightGBM 1.0000 0:00:21 0.9918 0.9941
 2 Ensemble 1.0000 0:00:29 0.9941 0.9941

#### Portal URL for Monitoring Runs

The following will provide a link to the web interface to explore individual run details and status.

In [27]:
displayHTML("<a href={} target='_blank'>Your experiment in Azure Portal: {}</a>".format(local_run.get_portal_url(), local_run.id))

Your experiment in Azure Portal: AutoML_f5fa6fa9-6a9f-4bd5-9fd2-b4132e4ac2be

## Retrieve the Best Model

Below we select the best pipeline from our iterations. The *get_output* method on automl_classifier returns the best run and the fitted model for the last *fit* invocation. There are overloads on *get_output* that allow you to retrieve the best run and fitted model for *any* logged metric or a particular *iteration*.

In [29]:
best_run, fitted_model = local_run.get_output()

In [30]:
fitted_model

Out[ 41 ]: 
Pipeline(memory=None,
 steps=[('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=None,
 estimators=[('LightGBM', Pipeline(memory=None,
 steps=[('MaxAbsScaler', MaxAbsScaler(copy=True)), ('LightGBMClassifier', <automl.client.core.common.model_wrappers.LightGBMClassifier object at 0x7f4085939c50>)]))],
 flatten_transform=None, weights=[1.0]))])

We can see from the above results that `StandardScalerWrapper` was used to scale the features and a `LightGBMClassifier` was chosen as the best model based on the metric we defined. This of course does NOT automatically also make it the best model in production, but choosing the right model for production is beyond the scope of this course so we will not address it here.

### Hands-on lab

Run the following cell and go to the link provided under Details Page. This links will take us to the Azure portal. Examine the content of the page. Can you find what resource group this resource is under? What kind of resource is it?

In [34]:
displayHTML("<a href={} target='_blank'>Your experiment in Azure Portal: {}</a>".format(best_run.get_portal_url(), best_run.id))

Your experiment in Azure Portal: AutoML_f5fa6fa9-6a9f-4bd5-9fd2-b4132e4ac2be_2

In addition to choosing a good algorithm, the experiment also tuned hyper-parameters. So our model didn't just run with the default hyper-parameter values. Find out how we can get the chosen hyper-parameters from the `fitted_model` object. Describe the hyper-parameters you see. Which ones do you think are the most critical ones?

In [36]:
# write solution here

### End of lab

We can run the following code to get the hyper-parameters for the chosen model.

In [39]:
scaler_params = fitted_model.steps[0][1].get_params()
scaler_params

Out[ 44 ]: 
{'LightGBM__LightGBMClassifier__colsample_bytree': 0.1988888888888889,
 'LightGBM__LightGBMClassifier__subsample_for_bin': 200000,
 'flatten_transform': None,
 'LightGBM__LightGBMClassifier__class_weight': None,
 'LightGBM__LightGBMClassifier__num_leaves': 119,
 'LightGBM__MaxAbsScaler': MaxAbsScaler(copy=True),
 'LightGBM__LightGBMClassifier__verbose': -10,
 'LightGBM__LightGBMClassifier__silent': True,
 'LightGBM__LightGBMClassifier__learning_rate': 0.06842421052631578,
 'estimators': [('LightGBM', Pipeline(memory=None,
 steps=[('MaxAbsScaler', MaxAbsScaler(copy=True)), ('LightGBMClassifier', <automl.client.core.common.model_wrappers.LightGBMClassifier object at 0x7f4085939c50>)]))],
 'LightGBM__LightGBMClassifier__boosting_type': 'gbdt',
 'LightGBM__LightGBMClassifier__subsample_freq': 0,
 'LightGBM__LightGBMClassifier__min_child_samples': 3764,
 'LightGBM__memory': None,
 'LightGBM__MaxAbsScaler__copy': True,
 'LightGBM__LightGBMClassifier__n_jobs': 1,
 'LightGBM__LightGBMClassifier__objective': None,
 'LightGBM__LightGBMClassifier': <automl.client.core.common.model_wrappers.LightGBMClassifier at 0x7f4085939c50>,
 'LightGBM__LightGBMClassifier__random_state': None,
 'LightGBM__LightGBMClassifier__n_estimators': 25,
 'LightGBM__LightGBMClassifier__min_child_weight': 4,
 'LightGBM__LightGBMClassifier__min_split_gain': 1,
 'LightGBM__LightGBMClassifier__subsample': 0.8910526315789474,
 'LightGBM__LightGBMClassifier__importance_type': 'split',
 'weights': [1.0],
 'LightGBM__steps': [('MaxAbsScaler', MaxAbsScaler(copy=True)),
 ('LightGBMClassifier',
 <automl.client.core.common.model_wrappers.LightGBMClassifier at 0x7f4085939c50>)],
 'LightGBM__LightGBMClassifier__reg_alpha': 0.8421052631578947,
 'LightGBM': Pipeline(memory=None,
 steps=[('MaxAbsScaler', MaxAbsScaler(copy=True)), ('LightGBMClassifier', <automl.client.core.common.model_wrappers.LightGBMClassifier object at 0x7f4085939c50>)]),
 'classification_labels': None,
 'LightGBM__LightGBMClassifier__max_bin': 130,
 'LightGBM__LightGBMClassifier__reg_lambda': 0.21052631578947367,
 'LightGBM__LightGBMClassifier__max_depth': 10}

In [40]:
model_params = fitted_model.steps[1][1].get_params()
model_params

--------------------------------------------------------------------------- 
 IndexError Traceback (most recent call last)
 <command-1871202421131484> in <module> () 
 ----> 1 model_params = fitted_model . steps [ 1 ] [ 1 ] . get_params ( ) 
 2 model_params 

 IndexError : list index out of range

## Score and evaluate the chosen model

Let's now pick the best model returned by the experiment and use it to get predictions for the test data. This is simply done by replacing `manual_model.predict` with `fitted_model.predict`.

In [43]:
y_pred = fitted_model.predict(X_test)

We should get the same confusion matrix we did in the section above.

In [45]:
confusion_matrix(y_test.values[:, 0], y_pred)

Out[ 47 ]: 
array([[62021, 1],
 [ 406, 172]])

We use `classification_report` to automatically calculate precision, recall, and the F-1 score from the confusion matrix above.

In [47]:
cl_report = classification_report(y_test.values[:, 0], y_pred)
print(cl_report)

precision recall f1-score support

 0 0.99 1.00 1.00 62022
 1 0.99 0.30 0.46 578

avg / total 0.99 0.99 0.99 62600

The AUC is just the area under the ROC curve shown here:

In [49]:
from sklearn.metrics import auc, roc_curve
fpr, tpr, thresholds = roc_curve(y_test.values[:, 0], y_pred)
roc_auc = auc(fpr, tpr)

import matplotlib.pyplot as plt
plt.plot(fpr, tpr, 'b', label = 'AUC = {0:.2f}'.format(roc_auc))
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()
display()

## Register fitted model for deployment

Now that we have a model we're happy with, we register it to our Azure ML account. This will be the first step toward model management and deployment, which we cover in the next Notebook. Registered models can also be loaded into other workspaces.

In [52]:
description = 'automated ML PdM (predict y_1)'
tags = None
model = local_run.register_model(description=description, tags=tags)
local_run.model_id # Use this id to deploy the model as a web service in Azur

Registering model AutoMLf5fa6fa96best
 Out[ 50 ]: 'AutoMLf5fa6fa96best'

Optionally, we can also take the model and save it on disk as a pickle file, as shown here:

In [54]:
from sklearn.externals import joblib
joblib.dump(value=fitted_model, filename='model.pkl')

Out[ 51 ]: ['model.pkl']

### Hands-on lab

<div class="alert alert-info">
So far we used automated ML to automatically train a classification model for only one of the four failure types, namely `y_1`. Write a loop to use automated ML to train, score and evaluate and finally register a classification model for `y_2`, `y_3` and `y_4`. In each case, compare the evaluation metrics we obtain to the ones we obtained in the previous Notebook when we hand-trained a model.
</div>

In [57]:
all_output = {'y_1': {'local_run': local_run, 'model': model, 'cl_report': cl_report}}

for i in range(1, 4): ## loop over each target class
    print("Running automl on y_" + str(i+1))
    ## YOUR CODE GOES HERE (modify the config file to point to the right target class)
    ## YOUR CODE GOES HERE (submit the automated ML experiment)
    ## YOUR CODE GOES HERE (extract the best model)
    ## YOUR CODE GOES HERE (predict on the test data)
    ## YOUR CODE GOES HERE (find the classification report for the best model and add it to y_report)
    ## YOUR CODE GOES HERE (add results to all_output)
    ## YOUR CODE GOES HERE (register the final model)

Here's the solution:

In [59]:
all_output = {'y_1': {'local_run': local_run, 'model': model, 'cl_report': cl_report}}

for i in range(1, 4): ## loop over each target class
    print("Running automl on y_" + str(i+1))
    automl_config.fit_params.update({'y': y_train.values[:, i]})
    local_run = experiment.submit(automl_config, show_output=True)
    best_run, fitted_model = local_run.get_output()
    y_pred = fitted_model.predict(X_test)
    cl_report = classification_report(y_test.values[:, i], y_pred)
    print(cl_report)
    all_output['y_' + str(i+1)] = {'local_run': local_run, 'cl_report': cl_report}
    description = 'autometed ML PdM (predict y_{})'.format(str(i+1))
    model = local_run.register_model(description=description, tags=None)

<div class="alert alert-info">
NOTE: We can use automated ML to directly train a multi-class classification model instead of training many binary classification models. We will explore that in another Notebook.
</div>

### End of lab

# The end

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.